In [40]:
%matplotlib inline

In [41]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
from tqdm import tqdm_notebook as tqdm

In [42]:
import pickle

In [43]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [45]:
SOS_token = 0
EOS_token = 1

In [46]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.MAX_LENGTH = 2
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        words = sentence.split(' ')
        self.MAX_LENGTH = 2+max(self.MAX_LENGTH-2,len(words))
        for word in words:
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
    
    def addWords(self, words):
        for word in words:
            addWord(word)

In [47]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    #s = re.sub(r"([.!?])", r" \1", s)
    #s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [48]:
def readLangs(articles_file, titles_file):
    print("Reading lines...")

    # Read the file and split into lines
    articles_lines = open(articles_file, encoding='utf-8').read().strip().split('\n')
    titles_lines = open(titles_file, encoding='utf-8').read().strip().split('\n')
    print("Files read")
    # Split every line into pairs and normalize
    articles_lines = [normalizeString(s) for s in articles_lines]
    titles_lines = [normalizeString(s) for s in titles_lines]

    pairs= []
    
    for i in range(len(articles_lines)):
        pair = [articles_lines[i],titles_lines[i]]
        pairs.append(pair)
    
    sent = Lang("sentence")
    summ = Lang("summary")

    return sent, summ, pairs

In [49]:
def prepareData(articles_file, titles_file):
    sent, summ, pairs = readLangs(articles_file,titles_file)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        sent.addSentence(pair[0])
        summ.addSentence(pair[1])
    print("Counted words:")
    print(sent.name, sent.n_words)
    print(summ.name, summ.n_words)
    return sent, summ, pairs

In [50]:
sent, summ, pairs = prepareData("data/train.article.txt","data/train.title.txt")
print(sent.MAX_LENGTH, summ.MAX_LENGTH)

Reading lines...
Files read
Read 3803957 sentence pairs
Counting words...
Counted words:
sentence 90676
summary 43841
101 49


In [51]:
with open("classes/sent", "wb") as output:
    pickle.dump(sent, output, pickle.HIGHEST_PROTOCOL)
with open("classes/summ", "wb") as output:
    pickle.dump(summ, output, pickle.HIGHEST_PROTOCOL)
with open("classes/pairs", "wb") as output:
    pickle.dump(pairs, output, pickle.HIGHEST_PROTOCOL)

In [52]:
'''
sent_file = open("classes/sent",'rb')
sent = pickle.load(sent_file)
summ_file = open("classes/summ",'rb')
summ = pickle.load(summ_file)
pairs_file = open("classes/pairs",'rb')
pairs = pickle.load(pairs_file)
'''

'\nsent_file = open("classes/sent",\'rb\')\nsent = pickle.load(sent_file)\nsumm_file = open("classes/summ",\'rb\')\nsumm = pickle.load(summ_file)\npairs_file = open("classes/pairs",\'rb\')\npairs = pickle.load(pairs_file)\n'

In [53]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [54]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [55]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=50):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [56]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(sent, pair[0])
    target_tensor = tensorFromSentence(summ, pair[1])
    return (input_tensor, target_tensor)

In [57]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0
        
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [58]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [59]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [60]:
def trainIters(encoder, decoder, n_iters, print_every=10000, plot_every=10000,save_every=10000,learning_rate=0.001):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
        
    criterion = nn.NLLLoss()

    for iter in tqdm(range(1, n_iters + 1)):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=sent.MAX_LENGTH)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
            
        if iter % save_every==0:
            torch.save(encoder.state_dict(),f"weights/encoder_{iter}.pth")
            torch.save(decoder.state_dict(),f"weights/decoder_{iter}.pth")


In [61]:
def evaluate(encoder, decoder, sentence, max_length):
    with torch.no_grad():
        input_tensor = tensorFromSentence(sent, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(summ.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [62]:
def evaluateRandomly(encoder, decoder, n=3, max_length=101):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0], max_length)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [63]:
def evaluateRaw(encoder, decoder, sample, short, max_length):
    print('>', sample)
    print('=', short)
    output_words, attentions = evaluate(encoder, decoder, sample, max_length)
    output_sentence = ' '.join(output_words)
    print('<', output_sentence)
    print('')

In [64]:
hidden_size = 256
encoder1 = EncoderRNN(sent.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, summ.n_words, dropout_p=0.1, max_length=sent.MAX_LENGTH).to(device)

In [66]:
trainIters(encoder1, attn_decoder1,1000000,print_every=100,save_every=10000,plot_every=1000000,learning_rate=0.01)
#trainIters(encoder1, attn_decoder1, 2, print_every=1,save_every=1)

RuntimeError: invalid argument 6: wrong matrix size at c:\programdata\miniconda3\conda-bld\pytorch_1533100020551\work\aten\src\thc\generic/THCTensorMathBlas.cu:477

In [ ]:
#encoder1.load_state_dict(torch.load("weights/encoder_300000.pth"))
#attn_decoder1.load_state_dict(torch.load("weights/decoder_300000.pth"))

In [ ]:
#evaluateRandomly(encoder1, attn_decoder1)